In [11]:
import yaml
import pandas as pd
import tapo_p110_measurement as p110
from octoprint_api import get_octoprint_response
from scipy.stats import qmc
from pyDOE3 import fracfact_by_res
from tapo_p110_measurement import p110_device
import os
from scipy.integrate import trapezoid
import pyDOE3
import numpy as np

In [12]:
# Read config
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [13]:
# Set up data recorders
# input("Please start the Raspberry recorder.\n\
#       Press enter to continue...")


energy_consumption_sensor = p110.p110_device(config["sensor"]["current"]["frequency"],
                                             config["sensor"]["current"]["username"],
                                             config["sensor"]["current"]["password"],
                                             config["sensor"]["current"]["ip"])

while True:
    operational,data=get_octoprint_response()
    print(operational)

    while not operational:
        operational,data=get_octoprint_response()
    state=data["state"]
    name=data["job"]["file"]["name"]

    # start measurement if the name changes otherwise let the measurement run
    if name != initial_name and state=="Printing":
        slicer_settings_name,filename_pre=name.rsplit('_',1) # slicer_settings_standard_filename.gcode --> slicer_settings_standard , filename.gcode
        filename_final,_ = os.path.splitext(filename_pre) # filename.gcode --> filename , .gcode
        energy_consumption_sensor.start(filename_final+".csv")
        initial_name=name
        print("Started measuring")
        
    
    elif state!="Printing":
        print ("Stopped measuring")
        energy_consumption_sensor.stop()
    
        
    else:
        pass
#for experiment in experiments:
# while not 


energy_consumption_sensor.start()


ValueError: invalid literal for int() with base 10: '129.13.235.157'

In [ ]:
config

{'experiments': {'parameter_bounds': {'infill_density': [20, 50],
   'printing_temperature': [205, 225],
   'build_plate_temperature': [50, 70],
   'layer_height': [0.1, 0.3],
   'print_speed': [50, 100]}},
 'database': None,
 'sensor': {'current': {'frequency': 1,
   'username': 'svenja.hauck@kit.edu',
   'password': 'testpw12',
   'ip': '129.13.235.157'}},
 'settings': {'debug': True, 'log_level': 'INFO'}}

In [ ]:
p110 = p110_device(config["sensor"]["current"]["username"],
                   config["sensor"]["current"]["password"],
                   config["sensor"]["current"]["ip"]
                   )

In [ ]:
p110.start('test.csv')

Energy recording started


c:\Users\anato\anaconda3\envs\KIT_3D\Lib\threading.py:1010: RuntimeWarning: coroutine 'p110_device.capture_data' was never awaited
  self._target(*self._args, **self._kwargs)


# Parameters sampling
___

In [ ]:
# Read config
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)
config

{'experiments': {'parameter_bounds': {'infill_density': [20, 50],
   'printing_temperature': [205, 225],
   'build_plate_temperature': [50, 70],
   'layer_height': [0.1, 0.3],
   'print_speed': [50, 100]}},
 'database': None,
 'sensor': {'current': {'frequency': 1,
   'username': 'svenja.hauck@kit.edu',
   'password': 'testpw12',
   'ip': '129.13.235.157'}},
 'settings': {'debug': True, 'log_level': 'INFO'}}

In [ ]:
bounds_values = list(config['experiments']['parameter_bounds'].values())
print(bounds_values)

[[20, 50], [205, 225], [50, 70], [0.1, 0.3], [50, 100]]


In [6]:
# LHS Samples
def get_lhs_points(bounds, n):
    dimension = len(bounds)

    sampler = qmc.LatinHypercube(dimension, scramble=False)
    samples = sampler.random(n)

    # Samples still scale with the bounds so that we can actually get usable parameter values
    lower_bound = [b[0] for b in bounds] 
    upper_bound = [b[1] for b in bounds]

    return qmc.scale(samples, lower_bound, upper_bound)

In [ ]:
lhs_points = get_lhs_points(bounds_values, 8)
print(lhs_points)
import pandas as pd
df_lhs=pd.DataFrame(lhs_points)

[[6.56250e-01 4.43750e+01 2.16875e+02 4.62500e+01 1.62500e-01 5.12500e+01]
 [7.68750e-01 3.68750e+01 2.24375e+02 5.62500e+01 2.12500e-01 7.37500e+01]
 [1.21875e+00 2.56250e+01 2.01875e+02 4.87500e+01 2.62500e-01 5.87500e+01]
 [8.81250e-01 4.06250e+01 2.28125e+02 5.37500e+01 1.37500e-01 8.12500e+01]
 [1.33125e+00 2.93750e+01 2.05625e+02 5.87500e+01 1.87500e-01 4.37500e+01]
 [1.44375e+00 4.81250e+01 2.13125e+02 4.12500e+01 1.12500e-01 6.62500e+01]
 [9.93750e-01 2.18750e+01 2.20625e+02 5.12500e+01 2.87500e-01 8.87500e+01]
 [1.10625e+00 3.31250e+01 2.09375e+02 4.37500e+01 2.37500e-01 9.62500e+01]]


In [ ]:
df_lhs.to_excel("lhs.xlsx")

In [ ]:
lhs_points.tolist()

[[0.76875, 36.875, 228.125, 48.75, 0.26249999999999996, 81.25],
 [0.65625, 33.125, 205.625, 51.25, 0.2125, 66.25],
 [0.88125, 48.125, 220.625, 46.25, 0.1375, 51.25],
 [1.33125, 40.625, 201.875, 56.25, 0.1875, 43.75],
 [1.10625, 21.875, 216.875, 43.75, 0.1625, 88.75],
 [0.9937499999999999, 25.625, 209.375, 53.75, 0.2375, 58.75],
 [1.44375, 29.375, 224.375, 58.75, 0.1125, 96.25],
 [1.21875, 44.375, 213.125, 41.25, 0.2875, 73.75]]

In [8]:
#FRAC FACT SAMPLES, means fractional factorial see
# The Pydoe3 Library and its functionality for the
# Creation of fractional-factorial designs.So it will be one
# Subsetted of all possible marginal values ​​used.(For more information see:
# https://pydoe3.readthedocs.io/en/latest/factorial.html#fractional-factorial)
def get_edge_points(bounds):
    parameters_to_test = []
    dimension = len(bounds)

    samples = fracfact_by_res(dimension, 3)

   # Select the bounds that correspond to the samples of the design (-1 means the lower and 1 upper limit)
# so that we can actually get usable parameter sets
    lower_bounds = [b[0] for b in bounds]
    upper_bounds = [b[1] for b in bounds]
    for sample in samples:
        edge_sample = []
        for i in range(dimension):
            if sample[i] == -1:
                edge_sample.append(lower_bounds[i])
            elif sample[i] == 1:
                edge_sample.append(upper_bounds[i])
        parameters_to_test.append(edge_sample)

    return parameters_to_test

In [ ]:
doe_points = get_edge_points(bounds_values)
print(doe_points)
df_doe=pd.DataFrame(doe_points)
df_doe.to_excel("doe.xlsx")

[[20, 205, 50, 0.3, 100], [50, 205, 50, 0.1, 50], [20, 225, 50, 0.1, 100], [50, 225, 50, 0.3, 50], [20, 205, 70, 0.3, 50], [50, 205, 70, 0.1, 100], [20, 225, 70, 0.1, 50], [50, 225, 70, 0.3, 100]]


___

In [ ]:
len(bounds_values)
bounds_values

[[20, 50], [200, 230], [40, 60], [0.1, 0.3], [40, 100]]

In [26]:
fracfact_by_res(5,3).shape
frac = fracfact_by_res(5,3)
np.all(pyDOE3.fracfact('a b c ab ac')==frac)

True

In [24]:
pyDOE3.fracfact('a b c ab ac')

array([[-1., -1., -1.,  1.,  1.],
       [ 1., -1., -1., -1., -1.],
       [-1.,  1., -1., -1.,  1.],
       [ 1.,  1., -1.,  1., -1.],
       [-1., -1.,  1.,  1., -1.],
       [ 1., -1.,  1., -1.,  1.],
       [-1.,  1.,  1., -1., -1.],
       [ 1.,  1.,  1.,  1.,  1.]])

# Compute energy consumption of a print
___

In [3]:
df = pd.read_csv("stringing2_impossiblecube.csv")
df

,current_power,local_time,month_energy,month_runtime,today_energy,today_runtime
0,75212,2024-07-03T10:45:03,3705,3525,182,645
1,75212,2024-07-03T10:45:04,3705,3525,182,645
2,84806,2024-07-03T10:45:06,3705,3525,182,645
3,84806,2024-07-03T10:45:07,3705,3525,182,645
4,84806,2024-07-03T10:45:08,3705,3525,182,645
...,...,...,...,...,...,...
145,24791,2024-07-03T10:48:00,3710,3528,187,648
146,24791,2024-07-03T10:48:01,3710,3528,187,648
147,24791,2024-07-03T10:48:02,3710,3528,187,648
148,24648,2024-07-03T10:48:03,3710,3528,187,648


In [14]:
df["local_time"] = pd.to_datetime(df["local_time"])
df.local_time

NameError: name 'df' is not defined

In [7]:
first_timestamp = df['local_time'].iloc[0]
df['timedelta_seconds'] = (df['local_time'] - first_timestamp).dt.total_seconds()
print(df.timedelta_seconds.unique())

[  0.   1.   3.   4.   5.   6.   7.   9.  10.  11.  12.  14.  15.  16.
  18.  19.  20.  21.  23.  24.  25.  26.  28.  29.  30.  31.  33.  34.
  35.  37.  38.  39.  40.  41.  43.  44.  45.  46.  48.  49.  50.  51.
  52.  54.  55.  56.  58.  59.  60.  61.  63.  64.  65.  66.  68.  69.
  70.  71.  73.  74.  75.  76.  78.  79.  80.  81.  82.  84.  85.  86.
  87.  89.  90.  91.  92.  93.  95.  96.  97.  98. 100. 102. 103. 104.
 106. 107. 108. 110. 111. 112. 113. 114. 116. 117. 118. 119. 121. 122.
 123. 124. 125. 127. 128. 129. 130. 131. 132. 133. 134. 136. 137. 138.
 140. 141. 142. 143. 144. 146. 147. 148. 149. 150. 151. 152. 154. 155.
 156. 157. 158. 159. 160. 162. 163. 164. 165. 166. 167. 168. 169. 170.
 172. 173. 174. 175. 176. 177. 178. 179. 180. 181.]


In [8]:
energy_in_mWs = trapezoid(df.current_power,df.timedelta_seconds)
energy_in_Wh = energy_in_mWs/(1e3*3600) # mWs -> Wh
print(energy_in_Wh)

4.680560694444444


In [9]:
rough_energy_estimation=df.current_power.sum()/(1e3*3600)
print(f'The print took around {round(energy_in_Wh,3)}Wh of energy, or {round(energy_in_Wh*3600,3)}J in {int(df.timedelta_seconds.iloc[-1])} seconds')

The print took around 4.681Wh of energy, or 16850.018J in 181 seconds


___